# CoDeepNEAT demo
## CS081 project checkpoint demo
### Harsha Uppli, Alan Zhao, Gabriel Meyer-Lee

The following notebook demonstrates using CoDeepNEAT to solve CIFAR-10

In [9]:
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
from math import pi, floor
from random import random
from codeepneat import codeepneat, config, population, chromosome, genome, visualize
import pickle
import numpy as np
import keras

### Problem: CIFAR10 data set

Conveniently, it's also built into Keras, which our CoDeepNEAT imiplementation is built off of.

In [10]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = y_train[:,0]
y_test = y_test[:,0]
print("data shapes")
print("  x train:", x_train.shape)
print("  x test:", x_test.shape)
print("  y train:", y_train.shape)
print("  y test:", y_test.shape)
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
data = [x_train, y_train, x_test, y_test]

data shapes
  x train: (50000, 32, 32, 3)
  x test: (10000, 32, 32, 3)
  y train: (50000,)
  y test: (10000,)


### Configuring NEAT

Many of the options and inputs are still handled through the config file. The config file has been shortened considerably as many parameters have been eliminated, although many parameters have also been introduced which could be added.

In [11]:
%%file configCIFAR10
#--- parameters for the robot experiment ---#
[phenotype]
input_nodes         = 32,32,3
output_nodes        = 10
conv                = True

[genetic]
max_fitness_threshold = 1

# Human reasoning
pop_size              = 10
prob_addconv          = 0.9
prob_addlayer         = 0.1
prob_addmodule        = 0.05
elitism               = 1

[genotype compatibility]
compatibility_threshold = 3.0
compatibility_change    = 0.0
excess_coeficient       = 1.0
disjoint_coeficient     = 1.0
connection_coeficient   = 0.4

[species]
species_size        = 10
survival_threshold  = 0.2
old_threshold       = 30
youth_threshold     = 10
old_penalty         = 0.2
youth_boost         = 1.2
max_stagnation      = 15

Overwriting configCIFAR10


In [12]:
epoch = 5
pop_module = 40
pop_blueprint = 10
pop_model = 10
num_generation = 5

### Fitness

For this demonstration we'll be using supervised learning to train the networks produced by CoDeepNEAT on NMIST and then use their accuracy after 10 epochs as their fitness.

In [13]:
def fitness(network, data):
    network.fit(data[0], data[1],  epochs=5)
    loss, acc = network.evaluate(data[2], data[3])
    return acc

### Evolution

Evolution with CoDeepNEAT is slightly different than evolution with NEAT. The main difference is coevolution, where we have two separate populations with a hierarchical relationship evolving together.

In [14]:
def evolve(n):
    config.load('configCIFAR10')
    # Create 2 separate populations (size is now defined explicitly, but config file can still be used)
    module_pop = population.Population(pop_module, chromosome.ModuleChromo)
    # As the top hierarchical level, the blueprint population needs to be able to see the module population
    blueprint_pop = population.Population(pop_blueprint, chromosome.BlueprintChromo, module_pop)
    # Most of the actual evolving is now handled outside of the population, by CoDeepNEAT
    # Instead of requiring the user to overwrite the evaluation function, CoDeepNEAT evaluates the populations itself,
    # it simply requires a fitness function for the networks it creates passed in as an argument.
    codeepneat.epoch(n, blueprint_pop, module_pop, pop_model, fitness, data, save_best=True, name='CIFAR10')
    # It will still stop if fitness surpasses the max_fitness_threshold in config file
    # Plots the evolution of the best/average fitness
    visualize.plot_stats(module_pop.stats, name="CIFAR10mod_")
    visualize.plot_stats(blueprint_pop.stats, name="CIFAR10bp_")
    # Visualizes speciation
    #visualize.plot_species(module_pop.species_log, name="NMISTmod_")
    #visualize.plot_species(blueprint_pop.species_log, name="NMISTbp_")

In [15]:
evolve(num_generation)

Epoch 1/5
50000/50000 [==============================] - 18s - loss: 1.6611 - acc: 0.4252    
Epoch 2/5
50000/50000 [==============================] - 17s - loss: 1.3870 - acc: 0.5135    
Epoch 3/5
50000/50000 [==============================] - 17s - loss: 1.2932 - acc: 0.5455    
Epoch 4/5
50000/50000 [==============================] - 17s - loss: 1.2260 - acc: 0.5717    
Epoch 5/5
50000/50000 [==============================] - 17s - loss: 1.6807 - acc: 0.4242    
Epoch 2/5
 9760/50000 [====>.........................] - ETA: 14s - loss: 1.4352 - acc: 0.4911

KeyboardInterrupt: 

In [29]:
def eval_best(chromo_file):
    config.load('configMNIST')
    fp = open(chromo_file, "rb")
    chromo = pickle.load(fp)
    print(chromo)
    fp.close()
    visualize.draw_net(chromo, "_" + chromo_file)    
    network = codeepneat.produce_net(chromo)
    network.fit(x_train, y_train, epochs=50)
    fitness = network.evaluate(x_test, y_test)
    print("fitness", fitness)

In [32]:
eval_best("NMISTbp_best_chromo_0")

FileNotFoundError: [Errno 2] No such file or directory: 'NMISTbp_best_chromo_0'